In [95]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [96]:
mnist_dataset,mnist_info = tfds.load(name="mnist",with_info=True,as_supervised=True)

In [97]:
mnist_train,mnist_test = mnist_dataset["train"],mnist_dataset["test"]

In [98]:
num_val = 0.1*mnist_info.splits["train"].num_examples
num_val = tf.cast(num_val,tf.int64)
num_test = mnist_info.splits["test"].num_examples
num_test = tf.cast(num_test,tf.int64)

In [99]:
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image,label

In [100]:
scaled_train_and_val_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [101]:
BUFFER_SIZE = 10000
shuffled_train_and_val_data = scaled_train_and_val_data.shuffle(BUFFER_SIZE)

In [102]:
validation_data = shuffled_train_and_val_data.take(num_val)
train_data = shuffled_train_and_val_data.skip(num_val)

In [103]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_val)
test_data = test_data.batch(num_test)

validation_inputs,validation_targets = next(iter(validation_data))

In [104]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.models.Sequential() # Build a model
model.add(tf.keras.layers.Flatten(input_shape=(28,28,1))) # Add input layer
model.add(tf.keras.layers.Dense(units=hidden_layer_size,activation="relu")) # Add first hidden layer
model.add(tf.keras.layers.Dense(units=hidden_layer_size,activation="relu")) # Add second hidden layer
model.add(tf.keras.layers.Dense(units=output_size,activation="softmax")) # Add output layer
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"]) # Compile the model

In [105]:
NUM_EPOCHS = 5

model.fit(x=train_data,validation_data=(validation_inputs,validation_targets),epochs=NUM_EPOCHS,verbose=2) # Fit model

Epoch 1/5
540/540 - 4s - loss: 0.2717 - accuracy: 0.9215 - val_loss: 0.1350 - val_accuracy: 0.9597
Epoch 2/5
540/540 - 2s - loss: 0.1075 - accuracy: 0.9676 - val_loss: 0.0881 - val_accuracy: 0.9732
Epoch 3/5
540/540 - 2s - loss: 0.0715 - accuracy: 0.9782 - val_loss: 0.0723 - val_accuracy: 0.9765
Epoch 4/5
540/540 - 2s - loss: 0.0541 - accuracy: 0.9835 - val_loss: 0.0662 - val_accuracy: 0.9785
Epoch 5/5
540/540 - 2s - loss: 0.0426 - accuracy: 0.9868 - val_loss: 0.0413 - val_accuracy: 0.9867


In [106]:
test_loss, test_accuracy = model.evaluate(test_data) # Test model

1/1 [==============================] - 1s 567ms/step - loss: 0.0737 - accuracy: 0.9780


In [107]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f} Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.07 Test accuracy: 97.80%
